## Load json file data using curl

In [11]:
! curl -L -o assessment-attempts-20180128-121051-nested.json https://goo.gl/ME6hjp

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 9096k  100 9096k    0     0  23.1M      0 --:--:-- --:--:-- --:--:-- 29.8M


## briefly check the content of the json file using jq pretty print

In [12]:
! cat assessment-attempts-20180128-121051-nested.json | jq '.[0]'

{
  "keen_timestamp": "1516717442.735266",
  "max_attempts": "1.0",
  "started_at": "2018-01-23T14:23:19.082Z",
  "base_exam_id": "37f0a30a-7464-11e6-aa92-a8667f27e5dc",
  "user_exam_id": "6d4089e4-bde5-4a22-b65f-18bce9ab79c8",
  "sequences": {
    "questions": [
      {
        "user_incomplete": true,
        "user_correct": false,
        "options": [
          {
            "checked": true,
            "at": "2018-01-23T14:23:24.670Z",
            "id": "49c574b4-5c82-4ffd-9bd1-c3358faf850d",
            "submitted": 1,
            "correct": true
          },
          {
            "checked": true,
            "at": "2018-01-23T14:23:25.914Z",
            "id": "f2528210-35c3-4320-acf3-9056567ea19f",
            "submitted": 1,
            "correct": true
          },
          {
            "checked": false,
            "correct": true,
            "id": "d1bf026f-554f-4543-bdd2-54dcf105b826"
          }
        ],
        "user_submitted": true,
        "id": "7a2ed6d3-f492-49b

## Launch docker container (using docker-compose file from course 7)

In [13]:
! docker-compose up -d

Creating network "project-2-ocychang_default" with the default driver
Creating project-2-ocychang_mids_1 ... 
Creating project-2-ocychang_cloudera_1 ... 
Creating project-2-ocychang_zookeeper_1 ... 
Creating project-2-ocychang_kafka_1     ... mdone
Creating project-2-ocychang_spark_1     ... mdone
ting project-2-ocychang_spark_1     ... done

## Create kafka topic to retrieve the json data

In [15]:
! docker-compose exec kafka \
   kafka-topics \
    --create \
    --topic assessments \
    --partitions 1 \
    --replication-factor 1 \
    --if-not-exists \
    --zookeeper zookeeper:32181

Created topic assessments.


In [16]:
! docker-compose exec kafka \
  kafka-topics \
    --describe \
    --topic assessments \
    --zookeeper zookeeper:32181

Topic: assessments	PartitionCount: 1	ReplicationFactor: 1	Configs: 
	Topic: assessments	Partition: 0	Leader: 1	Replicas: 1	Isr: 1


## Publish json data into kafka topic: assessments

In [17]:
! docker-compose exec mids \
  bash -c "cat ~/project-2-ocychang/assessment-attempts-20180128-121051-nested.json \
    | jq '.[]' -c \
    | kafkacat -P -b kafka:29092 -t assessments && echo 'Json messages published.'"

Json messages published.


## Start Spark

In [18]:
! docker-compose exec spark env PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port 8888 --ip 0.0.0.0 --allow-root --notebook-dir=/w205/' pyspark

[I 05:40:38.299 NotebookApp](B Writing notebook server cookie secret to /root/.local/share/jupyter/runtime/notebook_cookie_secret
[I 05:40:38.327 NotebookApp](B Serving notebooks from local directory: /w205
[I 05:40:38.327 NotebookApp](B 0 active kernels 
[I 05:40:38.327 NotebookApp](B The Jupyter Notebook is running at: http://0.0.0.0:8888/?token=39ccc4100281bbe0b8ac45e32f0119a1075d7dcad6f398cf
[I 05:40:38.327 NotebookApp](B Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
[C 05:40:38.328 NotebookApp] 
    
    Copy/paste this URL into your browser when you connect for the first time,
    to login with a token:
        http://0.0.0.0:8888/?token=39ccc4100281bbe0b8ac45e32f0119a1075d7dcad6f398cf
[I 05:40:54.228 NotebookApp](B 302 GET /?token=39ccc4100281bbe0b8ac45e32f0119a1075d7dcad6f398cf (73.34.84.102) 0.72ms
[I 05:40:59.145 NotebookApp](B Writing notebook-signing key to /root/.local/share/jupyter/notebook_secret
[W 05:40:59.147 Notebook

In [8]:
from requests import get
ip = get('https://api.ipify.org').text
print ('IP is:', ip)

IP is: 35.230.20.101


In [23]:
# http://35.230.20.101:8888/?token=39ccc4100281bbe0b8ac45e32f0119a1075d7dcad6f398cf

In [22]:
#! docker-compose down